# Treinamento.
Como vimos na análise exploratória, uma árvore de decisão que considere apenas uma variável e tenha uma profundidade (parâmetro "max_depth") de dois níveis já é capaz de alcançar resultados razoáveis. Neste documento vamos treinar árvores de decisão com várias profundidades e que se utilizem de todas as variáveis disponíveis no conjunto de dados, com exceção das que apresentam variância nula. A expectativa é encontrar uma árvore de decisão simples que otimize a sensibilidade das previsões.

In [1]:
from pickle import dump
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
import eda_tools

data = pd.read_csv("./data/train.csv")
data.columns = data.columns.str.lower()

X = data.drop(columns=["id", "target"])
y = data["target"]

pipeline = Pipeline(
    [
        ("var", VarianceThreshold()),
        ("tree", DecisionTreeClassifier(class_weight="balanced")),
    ]
)

param_grid = {
    "tree__max_depth": range(2, 10),
}

scoring = {
    "sensitivity": make_scorer(recall_score),
    "specificity": make_scorer(eda_tools.specificity_score),
    "roc_auc": make_scorer(roc_auc_score),
}

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    scoring=scoring,
    n_jobs=-1,
    refit="roc_auc",
)

grid_search.fit(X, y)


GridSearchCV(estimator=Pipeline(steps=[('var', VarianceThreshold()),
                                       ('tree',
                                        DecisionTreeClassifier(class_weight='balanced'))]),
             n_jobs=-1, param_grid={'tree__max_depth': range(2, 10)},
             refit='roc_auc',
             scoring={'roc_auc': make_scorer(roc_auc_score),
                      'sensitivity': make_scorer(recall_score),
                      'specificity': make_scorer(specificity_score)})

Os resultados mostram que com uma simples árvore de decisão já somos capazes de alcançar uma área sob a curva ROC de 0,76, com sensibilidade e especificidade comparáveis.

In [2]:
results = pd.DataFrame(grid_search.cv_results_).sort_values(by="rank_test_roc_auc")

print("Melhor árvore encontrada:")
results[
    ["params", "mean_test_sensitivity", "mean_test_specificity", "mean_test_roc_auc"]
].iloc[0, :]


Melhor árvore encontrada:


params                   {'tree__max_depth': 5}
mean_test_sensitivity                  0.755643
mean_test_specificity                  0.757821
mean_test_roc_auc                      0.756732
Name: 3, dtype: object

A estrutura da árvore de decisão é um pouco grande demais para ser visualizada de maneira conveniente. Apesar disso, analisando a importância das variáveis segundo a árvore, vemos que de fato as variáveis "var15" (que associamos à idade) e "saldo_var30" são de longe as mais relevantes para a classificação dos clientes.

In [3]:
tree = grid_search.best_estimator_.named_steps["tree"]
selector = grid_search.best_estimator_.named_steps["var"]
X_cols = X.columns[selector.get_support()]

feature_importances = pd.DataFrame(
    {"importance": tree.feature_importances_, "feature": X_cols}
).sort_values(by="importance", ascending=False, ignore_index=True)

feature_importances = feature_importances[feature_importances["importance"] != 0]

feature_importances


,importance,feature
0,0.468079,var15
1,0.303962,saldo_var30
2,0.064634,saldo_var5
3,0.053460,saldo_medio_var5_hace2
4,0.031578,var38
5,0.014995,imp_op_var41_efect_ult1
6,0.011111,imp_op_var39_ult1
7,0.008575,num_var45_hace3
8,0.006796,saldo_medio_var5_ult1
9,0.006404,num_var22_ult3


Enviando o arquivo com as previsões para o Kaggle, a área sob a curva ROC no conjunto de teste foi de 0,74747. Esse resultado está dentro da meta estabelecidade no início deste projeto, de forma que ele pode ser considerado um sucesso.

In [4]:
test_data = pd.read_csv("./data/test.csv")

X_test = test_data.drop(columns=["ID"])

predictions = grid_search.best_estimator_.predict(X_test)

submission = pd.concat([test_data["ID"], pd.Series(predictions, name="TARGET")], axis=1)

submission.to_csv(
    "submission.csv",
    index=False,
)


Serializando o modelo para uso futuro sem a necessidade de recalculá-lo.

In [5]:
with open("modelo_treinado.pkl", "wb") as file:
    dump(tree, file)
